# Regress Out Tutorial

Author: [Yiming Yang](https://github.com/yihming)<br />
Date: 2020-09-01<br />
Notebook Source: [regress_out.ipynb](https://raw.githubusercontent.com/klarman-cell-observatory/pegasus/master/notebooks/regress_out.ipynb)

This tutorial shows how to regress out cell cycle using Pegasus. The dataset used contains human bone marrow data collected from 8 donors. It's stored at https://storage.googleapis.com/terra-featured-workspaces/Cumulus/MantonBM_nonmix_subset.zarr.zip.

## Cell-Cycle Scores

First, load the data:

In [ ]:
import pegasus as pg

data = pg.read_input("MantonBM_nonmix_subset.zarr.zip")
data

We need to do some preprocessing work before calculating the cell-cycle scores:

In [ ]:
pg.qc_metrics(data, percent_mito=10)
pg.filter_data(data)
pg.identify_robust_genes(data)
pg.log_norm(data)

Now calculate cell-cycle scores using `calc_signature_score` (see [here](https://pegasus.readthedocs.io/en/latest/api/pegasus.calc_signature_score.html) for its documentation):

In [ ]:
pg.calc_signature_score(data, 'cell_cycle_human')

By setting `cell_cycle_human`, Pegasus uses the cell cycle genes defined in [[Tirosh et al. 2015]](https://science.sciencemag.org/content/352/6282/189) for calculation. We can fetch these genes from Pegasus pre-stored file (https://raw.githubusercontent.com/klarman-cell-observatory/pegasus/master/pegasus/data_files/cell_cycle_human.gmt), as we'll need them in the following sections:

In [ ]:
cell_cycle_genes = []
with open("cell_cycle_human.gmt", 'r') as f:
    for line in f:
        cell_cycle_genes += line.strip().split('\t')[2:]

Moreover, the predicted phases of cells are stored in `data.obs` field with key `predicted_phase`:

In [ ]:
data.obs['predicted_phase'].value_counts()

And the corresponding scores are in `data.obs['G1/S']` and `data.obs['G2/M']`.

## Cell Cycle Effects

The presence of cell cycle effects can be shown in terms of Principal Components (PC), which are calculated from gene-count matrix with respect to cell cycle genes:

In [ ]:
data_cc_genes = data[:, cell_cycle_genes].copy()
pg.pca(data_cc_genes)
data.obsm['X_pca'] = data_cc_genes.obsm['X_pca']

After achieving PCA result, we can generate a scatter plot of first two PCs with predicted phase being the colors for cells:

In [ ]:
pg.scatter(data, attrs='predicted_phase', basis='pca', dpi=130)

In this plot, we can see that cells are quite separated due to their phases when cell cycle genes are considered.

## Regress Out Cell Cycle Effects

This section shows how to regress out cell cycle effects.

Pegasus performs this regressing out on PCs. After getting the original PCA matrix, it uses the cell-cycle scores for this operation:

In [ ]:
pca_key = pg.pc_regress_out(data, attrs=['G1/S', 'G2/M'])

When `pc_regress_out` is finished, the resulting new PCA matrix is stored in `data.obsm['X_pca_regressed']`, with the representation key `'pca_regressed'` returned to variable `pca_key`.

Then in downstream analysis, you should assign either `pca_key` or `'pca_regressed'` to `rep` argument of Pegasus functions whenever applicable.

Now let's see how PC scatter plot looks on the new PCs:

In [ ]:
pg.scatter(data, attrs=['predicted_phase'], basis=pca_key, dpi=130)

You can see that after regressing out cell-cycle scores, no evident cell-cycle effects are observed.